## 초기 system prompt 설정 및 벡터 store 생성

In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from random import randint, choice

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from openai import OpenAI
from langchain.vectorstores import Chroma
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()

permission = "frontend"

# 벡터 DB 세팅
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = Chroma(persist_directory=f"./chroma_db/{permission}", embedding_function=embedding_model)

C:\Users\miny0\AppData\Local\Temp\ipykernel_24732\1803320750.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
C:\Users\miny0\AppData\Local\Temp\ipykernel_24732\1803320750.py:19: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=f"./chroma_db/{permission}", embed

## 초기 질문 리스트 가져오기

In [2]:
import json

def load_questions_from_jsonl_minimal(path: str, key: str = "question"):
    """JSONL에서 각 라인의 `question` 값만 순서/중복/공백 그대로 추출"""
    questions = []
    with open(path, "r", encoding="utf-8-sig") as f:
        for line in f:
            line = line.strip()
            obj = json.loads(line)
            if isinstance(obj, dict) and key in obj:
                questions.append(obj[key])
    return questions

# 사용 예시
initial_questions = load_questions_from_jsonl_minimal(f"./company_questions_{permission}.jsonl")
initial_questions

["빌드/배포 가이드의 '1.1 환경 설정' 파트에서 Node.js 설치에 대한 구체적인 버전 요구사항은 무엇인가요?",
 "빌드/배포 가이드의 '1.2 프로젝트 클론' 파트에서 GitHub에서 프로젝트를 클론할 때 사용하는 명령어는 무엇인가요?",
 "빌드/배포 가이드의 '1.3 의존성 설치' 파트에서 의존성을 설치하기 위해 실행해야 하는 명령어는 무엇인가요?",
 "빌드/배포 가이드의 '2.1 빌드 명령어 실행' 파트에서 빌드를 실행하기 위해 사용할 수 있는 명령어는 무엇인가요?",
 "빌드/배포 가이드의 '2.2 빌드 결과 확인' 파트에서 빌드 결과물이 생성되는 폴더의 이름은 무엇인가요?",
 "빌드/배포 가이드의 '3.1 배포 환경 설정' 파트에서 배포할 서버에 접근하기 위한 방법은 무엇인가요?",
 "빌드/배포 가이드의 '3.2 파일 전송' 파트에서 빌드된 파일을 서버에 전송하기 위해 사용할 수 있는 방법은 무엇인가요?",
 "빌드/배포 가이드의 '3.3 서버 설정' 파트에서 웹 서버의 설정 파일을 수정할 때 확인해야 할 사항은 무엇인가요?",
 "빌드/배포 가이드의 '4.1 서비스 재시작' 파트에서 웹 서버를 재시작하기 위해 사용할 수 있는 명령어는 무엇인가요?",
 "빌드/배포 가이드의 '4.2 배포 확인' 파트에서 배포된 URL을 열어 확인해야 할 사항은 무엇인가요?",
 "빌드/배포 가이드의 '5.1 로그 확인' 파트에서 오류 메시지를 파악하기 위해 확인해야 하는 로그 파일의 경로는 무엇인가요?",
 "빌드/배포 가이드의 '5.2 문제 해결' 파트에서 발생한 오류를 해결하기 위해 어떤 방법을 사용할 수 있나요?",
 "빌드/배포 가이드의 '6. 문서화' 파트에서 문서화해야 할 내용은 무엇인가요?",
 "빌드/배포 가이드의 '다음 개정 제안'에서 제안된 배포 자동화 도구에 대한 추가 정보는 어떤 내용을 포함해야 하나요?",
 "빌드/배포 가이드의 '다음 개정 제안'에서 각 단계에 대한 예시를 추가하는 것이 왜 중요한가요?",
 "UI/UX 디자인

In [3]:
def question_chain_setting():
    llm = ChatOpenAI(model="gpt-4.1", temperature=0)

    basic_prompt = PromptTemplate.from_template(
        """
            다음 제공되는 대화 내역을 기반으로 다음에 나올 수 있는 사용자가 궁금해할 만한 후속 질문 하나를 사용자 입장에서 질문하세요.

            대화 내역: {history}

            절대로 AI입장에서의 질문을 만들지 마세요.
            ex) 추가적인 질문이 있으시면 언제든지 말씀해 주시기 바랍니다.
            ex) 추가적으로 궁금한 사항이 있으신가요?
            ex) 데이터 접근 권한 관리 방침에 대해 더 궁금한 사항이 있으면 말씀해 주세요.
            ex) 안녕하세요. 다른 궁금한 점이 있으신가요?
            ex) 안녕하십니까? 도움이 필요하신 부분이 있으신가요?
            ex) ~~ 예시는 다음과 같습니다.


            답변형태는 다음과 같이 해주세요.
            ex) 빌드 결과물의 확인 방법이나 배포 과정에 대해 알고 싶어.
            ex) 문서화 시 유의사항 알려줘.
            ex) 빌드/배포 가이드의 '6. 문서화' 파트에서 문서화해야 할 내용은 뭐야?
            ex) 모델 개발 & 성능 보고서에서 사용된 프로그래밍 언어는 무엇이야?

            <추가 지시사항>
            이전에 나왔던 질문은 최대한 만들지 말고, 연결된 질문으로 조금이라도 다른 질문을 만들어주세요.

        """
    )

    question_chain = basic_prompt | llm | StrOutputParser()

    return question_chain

In [4]:
question_chain = question_chain_setting()

In [6]:
# tool_prompt = f"사용자는 {permission}팀이며 반드시 {permission}_search 툴을 호출하세요."
tool_prompt = f"사용자는 {permission}(프론트엔드)팀에 속한 팀원입니다. 질문이 들어오면 반드시 {permission}_search 툴을 호출하여 문서를 검색하세요."

system_message = f"""
당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 한국인 AI 비서입니다.
다음 지침을 따르세요:
1. 기존의 말투는 잊고 정중하고 사무적인 어조로 답변해야 하세요.
2. 대화 내역의 말투도 참고하지 말고 무조건 정중하고 사문적인 어조로 답변하세요
3. 사실에 기반한 정보를 사용하세요.
4. 사용자의 질문에 대한 답변을 문서에서 찾을 수 없을 경우, "잘 모르겠습니다"라고 솔직하게 말하세요.
5. 사용자가 문서에 대한 질문이 아닌, "안녕"과 같은 일상적인 질문을 한다면 해당 내용에 대해서 적절히 답변해주세요.
6. 답변이 너무 길지 않게 하세요.
7. 사용자가 편하게 반말로 물어보더라도, 반드시 정중하고 사무적인 어조로 답변해야 합니다.
8. {tool_prompt}
"""

SYSTEM_PROMPT = {
    "role": "system",
    "content": system_message
}

def generate_one_chat_sample(initial_q: str, max_turns: int = 5):
    messages = [SYSTEM_PROMPT]
    current_q = initial_q
    for turn in range(randint(2, max_turns + 1)):
        # (1) User 질문 추가
        messages.append({"role": "user", "content": current_q})
        print(current_q)

        tool_call = f"<tool_call>{{\"name\": \"{permission}_search\", \"arguments\": {{\"keyword\": \"{current_q}\"}}}}</tool_call>"

        messages.append({"role": "assistant", "content": tool_call})

        # (2) 문서 검색 (vectorstore에서)
        docs = vectorstore.similarity_search(current_q, k=4)
        ref_text = "\n".join([f"{doc.page_content} [[ref{idx+1}]]" for idx, doc in enumerate(docs)])
        ref_text = f"검색 결과:\n-----\n{ref_text}"
        tool_res = f"<tool_response>{ref_text}</tool_response>"
        messages.append({"role": "user", "content": tool_res})

        # (3) assistant 답변 (GPT로 RAG)
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0
        )
        assistant_reply = response.choices[0].message.content
        messages.append({"role": "assistant", "content": assistant_reply})

        # 다음 질문 생성 여부 랜덤 (5턴이 max면 4턴 미만일때만 질문 생성)
        if turn < max_turns - 1 and choice([True, False]):

            followup = question_chain.invoke({
                "history": messages
            })
            current_q = followup
        else:
            break

    return {"messages": messages}


In [8]:
import time
from tqdm import tqdm

train_dataset = []
for q in tqdm(initial_questions):
    time.sleep(1)  # 1초 대기
    sample = generate_one_chat_sample(q, max_turns=4)
    train_dataset.append(sample)

# 저장
with open(f"qwen3_company_train_dataset_{permission}2.json", "w", encoding="utf-8") as f:
    json.dump(train_dataset, f, ensure_ascii=False, indent=2)


  0%|          | 0/275 [00:00<?, ?it/s]

빌드/배포 가이드의 '1.1 환경 설정' 파트에서 Node.js 설치에 대한 구체적인 버전 요구사항은 무엇인가요?
npm과 yarn 중 어떤 패키지 매니저를 사용하는 것이 더 권장되는지 알려줄 수 있어?


  0%|          | 1/275 [00:08<38:58,  8.53s/it]

빌드/배포 가이드의 '1.2 프로젝트 클론' 파트에서 GitHub에서 프로젝트를 클론할 때 사용하는 명령어는 무엇인가요?


  1%|          | 2/275 [00:12<25:53,  5.69s/it]

...

 99%|█████████▉| 273/275 [48:24<00:17,  8.55s/it]

API와의 연동 테스트는 어떤 테스트 종류에서 수행되나요?
API 연동 테스트를 자동화하려면 어떤 도구나 프레임워크를 사용하는 게 좋을까?
API 연동 테스트 자동화 시 발생할 수 있는 일반적인 오류 사례와 그 해결 방법도 안내해줄 수 있어?


100%|█████████▉| 274/275 [48:42<00:11, 11.24s/it]

테스트 결과를 문서화하는 이유는 무엇인가요?
테스트 결과를 문서화할 때 어떤 형식이나 항목을 포함해야 하는지 알려줘.
테스트 결과를 문서화한 후 팀원들과 효과적으로 공유하는 방법이 궁금해.


100%|██████████| 275/275 [48:59<00:00, 10.69s/it]
